In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import plotly.graph_objects as go

torch.manual_seed(0)

In [2]:
class NeuralNet(nn.Module):

    def __init__(self, input_dimension, output_dimension, n_hidden_layers, neurons, regularization_param, regularization_exp, retrain_seed):
        super(NeuralNet, self).__init__()
        # Number of input dimensions n
        self.input_dimension = input_dimension
        # Number of output dimensions m
        self.output_dimension = output_dimension
        # Number of neurons per layer
        self.neurons = neurons
        # Number of hidden layers
        self.n_hidden_layers = n_hidden_layers
        # Activation function
        self.activation = nn.Tanh()
        self.regularization_param = regularization_param
        # Regularization exponent
        self.regularization_exp = regularization_exp
        # Random seed for weight initialization

        self.input_layer = nn.Linear(self.input_dimension, self.neurons)
        self.hidden_layers = nn.ModuleList([nn.Linear(self.neurons, self.neurons) for _ in range(n_hidden_layers - 1)])
        self.output_layer = nn.Linear(self.neurons, self.output_dimension)
        self.retrain_seed = retrain_seed
        # Random Seed for weight initialization
        self.init_xavier()

    def forward(self, x):
        # The forward function performs the set of affine and non-linear transformations defining the network
        # (see equation above)
        x = self.activation(self.input_layer(x))
        for k, l in enumerate(self.hidden_layers):
            x = self.activation(l(x))
        return self.output_layer(x)

    def init_xavier(self):
        torch.manual_seed(self.retrain_seed)

        def init_weights(m):
            if type(m) == nn.Linear and m.weight.requires_grad and m.bias.requires_grad:
                g = nn.init.calculate_gain('tanh')
                torch.nn.init.xavier_uniform_(m.weight, gain=g)
                # torch.nn.init.xavier_normal_(m.weight, gain=g)
                m.bias.data.fill_(0)

        self.apply(init_weights)

    def regularization(self):
        reg_loss = 0
        for name, param in self.named_parameters():
            if 'weight' in name:
                reg_loss = reg_loss + torch.norm(param, self.regularization_exp)
        return self.regularization_param * reg_loss

class DrivenOscillatorPinns:
    def __init__(self, n_int_):
        self.n_int = n_int_


        # Initial condition to solve driven oscillator
        self.initial_x = 5
        self.initial_v = 0
        self.init_cond = torch.tensor([self.initial_x,self.initial_v])
        
        
        self.omega_o = 1
        self.omega_d = 1
        self.k = 1
        self.gamma   = 0.01
        self.mass = 1
        self.F_o = 0
        
        self.initial_weight  = 10
        self.residual_weight = 1
        self.trivial_killer_weight = 1
        
        # Extrema of the solution domain (t) in [0,5]
        self.domain_extrema = torch.tensor([[0, 10]])  
        
        # Generator of Sobol sequences
        self.soboleng = torch.quasirandom.SobolEngine(dimension=1)
        

        self.training_set_int = self.assemble_datasets()

        # F Dense NN to approximate the solution of the underlying heat equation
        self.approximate_solution = NeuralNet(input_dimension=self.domain_extrema.shape[0], output_dimension=1,
                                            n_hidden_layers=10,
                                            neurons=200,
                                            regularization_param=0.,
                                            regularization_exp=5,
                                            retrain_seed=20)
        
        

    ################################################################################################
    # Function to linearly transform a tensor whose value are between 0 and 1
    # to a tensor whose values are between the domain extrema
    def convert(self, tens):
        assert (tens.shape[1] == self.domain_extrema.shape[0])
        return tens * (self.domain_extrema[0][1] - self.domain_extrema[0][0]) + self.domain_extrema[0][0]

    def driving_term(self, t):
        return self.F_o*torch.cos(self.omega_d*t)/self.mass
    
    def exact_solution(self, t):
        return
    
    def add_interior_points(self):
        input_int = self.convert(self.soboleng.draw(self.n_int))
        output_int = torch.zeros((input_int.shape[0], 1))
        return input_int, output_int

    # Function returning the training sets S_sb, S_tb, S_int as dataloader
    def assemble_datasets(self):
        input_int, output_int = self.add_interior_points()         # S_int
        training_set_int = DataLoader(torch.utils.data.TensorDataset(input_int, output_int), batch_size=self.n_int, shuffle=False)

        return training_set_int
    

    ################################################################################################
    # Function to compute the terms required in the definition of the TEMPORAL boundary residual
    def initial_velocity(self,init_t):
        init_t.requires_grad = True
        pred_x_init = self.approximate_solution(init_t)
        pred_v_init = torch.autograd.grad(pred_x_init.sum(), init_t, create_graph=True)[0]
        
        return pred_v_init
    
    def initial_position(self,init_t):

        pred_x_init = self.approximate_solution(init_t)
        
        return pred_x_init

    # Function to compute the PDE residuals
    def compute_pde_residual(self, input_int):
        input_int.requires_grad = True
        u = self.approximate_solution(input_int)

        # grad compute the gradient of a "SCALAR" function L with respect to some input nxm TENSOR Z=[[x1, y1],[x2,y2],[x3,y3],...,[xn,yn]], m=2
        # it returns grad_L = [[dL/dx1, dL/dy1],[dL/dx2, dL/dy2],[dL/dx3, dL/dy3],...,[dL/dxn, dL/dyn]]
        # Note: pytorch considers a tensor [u1, u2,u3, ... ,un] a vectorial function
        # whereas sum_u = u1 + u2 + u3 + u4 + ... + un as a "scalar" one

        # In our case ui = u(xi), therefore the line below returns:
        # grad_u = [[dsum_u/dx1, dsum_u/dy1],[dsum_u/dx2, dsum_u/dy2],[dsum_u/dx3, dL/dy3],...,[dsum_u/dxm, dsum_u/dyn]]
        # and dsum_u/dxi = d(u1 + u2 + u3 + u4 + ... + un)/dxi = d(u(x1) + u(x2) u3(x3) + u4(x4) + ... + u(xn))/dxi = dui/dxi
        grad_u_t = torch.autograd.grad(u.sum(), input_int, create_graph=True)[0]
        grad_u_tt = torch.autograd.grad(grad_u_t.sum(), input_int, create_graph=True)[0]

        #grad_u_sq_x = torch.autograd.grad(u_sq.sum(), input_int, create_graph=True)[0][:,1]

        residual = self.mass * grad_u_tt  + self.k*u # - self.driving_term(input_int) + self.gamma*grad_u_t
        return residual.reshape(-1, )

    # Function to compute the total loss (weighted sum of spatial boundary loss, temporal boundary loss and interior loss)
    def compute_loss(self, inp_train_int ,verbose=True):
        
        init_t = torch.zeros(1)
        pred_init_position = self.initial_position(init_t)
        pred_init_velocity = self.initial_velocity(init_t)
        

        inp_train_int.requires_grad = True
        u = self.approximate_solution(inp_train_int)

        grad_u_t = torch.autograd.grad(u.sum(), inp_train_int, create_graph=True)[0]
        grad_u_tt = torch.autograd.grad(grad_u_t.sum(), inp_train_int, create_graph=True)[0]


        residual = self.mass * grad_u_tt  + self.omega_o**2*u  - self.driving_term(inp_train_int) + self.gamma*grad_u_t
        
        
        init_position_error = self.init_cond[0] - pred_init_position
        init_velocity_error = self.init_cond[1] - pred_init_velocity
        
        loss_initial_position = torch.mean(init_position_error ** 2)*self.initial_weight
        loss_initial_velocity = torch.mean(init_velocity_error ** 2)*self.initial_weight

        loss_int = torch.mean(residual ** 2)*self.residual_weight

        loss = loss_initial_position + loss_initial_velocity + loss_int
        
        if verbose: print("Total loss: ", round(loss.item(), 4), "| PDE Loss: ", round(loss_int.item(), 4), "| Position Loss: ", round(loss_initial_position.item(), 4),"| Velocity Loss: ", round(loss_initial_velocity.item(), 4))

        return loss + (torch.reciprocal(torch.mean(u ** 2)) + torch.reciprocal(torch.mean(grad_u_t ** 2)))*self.trivial_killer_weight 

    ################################################################################################
    def fit(self, num_epochs, optimizer, verbose=True):
        history = list()

        # Loop over epochs
        for epoch in range(num_epochs):
            if verbose: print("################################ ", epoch, " ################################")

            
            for j, (inp_train_int, u_train_int) in enumerate(self.training_set_int):
                def closure():
                    optimizer.zero_grad()
                    loss = self.compute_loss(inp_train_int ,verbose=verbose)
                    loss.backward()
                    
                    history.append(loss.item())
                    return loss

                optimizer.step(closure=closure)

        print('Final Loss: ', history[-1])

        return history




In [3]:
n_int = 1000
pinn = DrivenOscillatorPinns(n_int)
input_int_, output_int_ = pinn.add_interior_points()


In [4]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=input_int_[:,0], y=output_int_[:,0], mode='markers',marker=dict(color='red',size=1), name='Interior points'))
fig.show()

In [59]:

optimizer_LBFGS = optim.LBFGS(pinn.approximate_solution.parameters(),
                            lr=float(0.000001),
                            max_iter=50000,
                            max_eval=50000,
                            history_size=150,
                            line_search_fn="strong_wolfe",
                              tolerance_change=1.0 * np.finfo(float).eps)
optimizer_ADAM = optim.Adam(pinn.approximate_solution.parameters(),
                            lr=float(0.0001))

In [60]:
pinn.trivial_killer_weight = 0.
optimizer_ADAM.lr = 0.000005

hist = pinn.fit(num_epochs=500,
                optimizer=optimizer_LBFGS,
                verbose=True)

################################  0  ################################
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0


Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
################################  1  ################################
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
Total loss:  0.0002 | PDE Loss:  0.0002 | Position Loss:  0.0 | Velocity Loss:  0.0
######

KeyboardInterrupt: 

In [52]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=hist))
# set dark theme
fig.update_layout(title='Loss function', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_dark')
fig.show()

In [61]:
inputs = pinn.soboleng.draw(1000)
inputs = pinn.convert(inputs)
output = pinn.approximate_solution(inputs)
fig = go.Figure()
fig.add_trace(go.Scatter(x=inputs.detach().numpy()[:,0], y=output.detach().numpy()[:,0], mode='markers', marker=dict(color="red"), name='PINN solution'))
fig.add_trace(go.Scatter(x=inputs.detach().numpy()[:,0], y=5*torch.cos(inputs[:,0]).detach().numpy(), mode='markers', marker=dict(color="lime",size=3), name='Exact solution'))
fig.update_layout(title='Exact solution vs PINN solution', xaxis_title='time', yaxis_title='position', template='plotly_dark')
fig.show()

In [9]:
# print initial velocity and position
init_t = torch.zeros(1)
pred_init_position = pinn.initial_position(init_t)
pred_init_velocity = pinn.initial_velocity(init_t)
print("Initial position: ", pred_init_position.item())
print("Initial velocity: ", pred_init_velocity.item())

Initial position:  4.989751815795898
Initial velocity:  -5.529634654521942e-05


In [63]:
print("Mean square error: ", np.mean((output.detach().numpy()[:,0]*torch.cos(inputs[:,0]).detach().numpy())**2))

Mean square error:  9.472619
